**API Data**

In [ ]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import time

# Dataset yükləndi
df = pd.read_csv('/content/drive/MyDrive/raw_data (1).csv')

# Unikal ölkələri tap
countries = df['Area'].unique()

# Nominatim obyektini yarat
geolocator = Nominatim(user_agent="crop_prediction_app_v2", timeout=10)

# Saniyədə maksimum 1 sorğu göndərmək üçün RateLimiter
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1.5)

country_coords = {}

print(f"{len(countries)} ölkə üçün koordinatlar tapılır...")

for country in countries:
    try:
        # Sorğu göndər
        location = geocode(country)
        if location:
            country_coords[country] = (location.latitude, location.longitude)
            print(f"Tapıldı: {country} -> {location.latitude}, {location.longitude}")
        else:
            country_coords[country] = (None, None)
            print(f"Tapılmadı: {country}")
    except Exception as e:
        print(f"Xəta baş verdi ({country}): {e}")
        country_coords[country] = (None, None)
        time.sleep(2) # Xəta olduqda fasile

# Koordinatlar datasetə əlavə olunur
df['lat'] = df['Area'].map(lambda x: country_coords.get(x, (None, None))[0])
df['lon'] = df['Area'].map(lambda x: country_coords.get(x, (None, None))[1])

print("Əməliyyat tamamlandı!")

101 ölkə üçün koordinatlar tapılır...
Tapıldı: Albania -> 41.000028, 19.9999619
Tapıldı: Algeria -> 28.0000272, 2.9999825
Tapıldı: Angola -> -11.8775768, 17.5691241
Tapıldı: Argentina -> -34.9964963, -64.9672817
Tapıldı: Armenia -> 40.7696272, 44.6736646
Tapıldı: Australia -> -24.7761086, 134.755
Tapıldı: Austria -> 47.59397, 14.12456
Tapıldı: Azerbaijan -> 40.3936294, 47.7872508
Tapıldı: Bahamas -> 24.7736546, -78.0000547
Tapıldı: Bahrain -> 26.1551249, 50.5344606
Tapıldı: Bangladesh -> 24.4769288, 90.2934413
Tapıldı: Belarus -> 53.4250605, 27.6971358
Tapıldı: Belgium -> 50.6402809, 4.6667145
Tapıldı: Botswana -> -23.1681782, 24.5928742
Tapıldı: Brazil -> -10.3333333, -53.2
Tapıldı: Bulgaria -> 42.6073975, 25.4856617
Tapıldı: Burkina Faso -> 12.0753083, -1.6880314
Tapıldı: Burundi -> -3.426449, 29.9324519
Tapıldı: Cameroon -> 4.6125522, 13.1535811
Tapıldı: Canada -> 61.0666922, -107.991707
Tapıldı: Central African Republic -> 7.0323598, 19.9981227
Tapıldı: Chile -> -31.7613365, -71.31

In [ ]:
!pip install openmeteo-requests requests-cache retry-requests

import openmeteo_requests

import pandas as pd
import requests_cache
from retry_requests import retry
import time
import numpy as np

# Cache + retry
cache_session = requests_cache.CachedSession(".cache", expire_after=-1)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

url = "https://archive-api.open-meteo.com/v1/archive"

# Yeni sütunlar
df["temperature_2m_yearly_avg"] = None
df["rain_yearly_total"] = None
df["snowfall_yearly_total"] = None

for idx, row in df.iterrows():

    if pd.isna(row["lat"]) or pd.isna(row["lon"]) or pd.isna(row["Year"]):
        continue

    year = int(row["Year"])

    params = {
        "latitude": row["lat"],
        "longitude": row["lon"],
        "start_date": f"{year}-01-01",
        "end_date": f"{year}-12-31",
        "daily": [
            "temperature_2m_mean",
            "rain_sum",
            "snowfall_sum"
        ],
        "timezone": "GMT"
    }

    try:
        responses = openmeteo.weather_api(url, params=params)
        daily = responses[0].Daily()

        temp = daily.Variables(0).ValuesAsNumpy()
        rain = daily.Variables(1).ValuesAsNumpy()
        snow = daily.Variables(2).ValuesAsNumpy()

        df.at[idx, "temperature_2m_yearly_avg"] = np.nanmean(temp)
        df.at[idx, "rain_yearly_total"] = np.nansum(rain)
        df.at[idx, "snowfall_yearly_total"] = np.nansum(snow)

        print(f"✔ Weather added: {row['Area']} | {year}")
        time.sleep(1)

    except Exception as e:
        print(f"❌ Error for {row['Area']} ({year}): {e}")

        if "limit exceeded" in str(e):
            print("Limit doldu, 60 saniyə gözlənilir...")
            time.sleep(60)
        else:
            time.sleep(2)

print("Weather data successfully appended!")


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df.head()

In [ ]:

df.to_csv("crop_yield_predict1.csv", index=False, encoding="utf-8-sig")

print("Fayl uğurla yadda saxlanıldı: crop_yield_predict1.csv")